<a href="https://colab.research.google.com/github/prathamchintamani/neuralnets/blob/main/iris_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [171]:
import pandas as pd
import torch
from torch import nn

import numpy as np

In [172]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [173]:
from sklearn.model_selection import train_test_split

In [174]:
from sklearn import datasets
import pandas as pd

# Load the Iris dataset
iris = datasets.load_iris()

# Create a DataFrame from the dataset
iris_df = pd.DataFrame(iris.data)
iris_df['class'] = iris.target
iris_df.columns = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']

In [175]:
df = iris_df.sample(frac = 1)
df_train,df_test = train_test_split(df,test_size = 0.1, random_state = 1)
y_train = df_train['class']
y_test = df_test['class']
X_train = df_train.drop('class', axis = 1)
X_test = df_test.drop('class', axis = 1)

In [176]:
y_train_t,y_test_t,X_train_t,X_test_t = torch.from_numpy(y_train.to_numpy()),torch.from_numpy(y_test.to_numpy()),torch.from_numpy(X_train.to_numpy()),torch.from_numpy(X_test.to_numpy())

In [177]:
y_train_t = y_train_t.type(torch.long).to(device)
y_test_t = y_test_t.type(torch.long).to(device)
X_train_t = X_train_t.type(torch.float32).to(device)
X_test_t = X_test_t.type(torch.float32).to(device)

In [178]:
class nn_model(nn.Module):
  def __init__(self):
    super().__init__()
    self.stack = nn.Sequential(
        nn.Linear(4,4),
        nn.ReLU(),
        nn.Linear(4,3)
    )

  def forward(self, X ):
    return self.stack(X)

In [179]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [180]:
model_0 = nn_model()

In [181]:
model_0.to(device)

nn_model(
  (stack): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=3, bias=True)
  )
)

In [182]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_0.parameters(), lr=0.01)

In [183]:
torch.manual_seed(42)
epochs = 1000


In [184]:
for epoch in range(epochs):
    ### Training
    model_0.train()

    # 1. Forward pass
    y_logits = model_0(X_train_t) # model outputs raw logits
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train_t)
    acc = accuracy_fn(y_true=y_train_t,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_0.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_0(X_test_t)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_test_t)
      test_acc = accuracy_fn(y_true=y_test_t,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 1.54913, Acc: 67.41% | Test Loss: 1.73177, Test Acc: 60.00%
Epoch: 10 | Loss: 1.07956, Acc: 67.41% | Test Loss: 1.18138, Test Acc: 60.00%
Epoch: 20 | Loss: 0.90943, Acc: 67.41% | Test Loss: 0.97641, Test Acc: 60.00%
Epoch: 30 | Loss: 0.84551, Acc: 67.41% | Test Loss: 0.89818, Test Acc: 60.00%
Epoch: 40 | Loss: 0.80704, Acc: 67.41% | Test Loss: 0.85681, Test Acc: 60.00%
Epoch: 50 | Loss: 0.77433, Acc: 67.41% | Test Loss: 0.82652, Test Acc: 66.67%
Epoch: 60 | Loss: 0.74393, Acc: 68.15% | Test Loss: 0.80040, Test Acc: 66.67%
Epoch: 70 | Loss: 0.71537, Acc: 70.37% | Test Loss: 0.77644, Test Acc: 66.67%
Epoch: 80 | Loss: 0.68863, Acc: 70.37% | Test Loss: 0.75401, Test Acc: 66.67%
Epoch: 90 | Loss: 0.66371, Acc: 70.37% | Test Loss: 0.73291, Test Acc: 66.67%
Epoch: 100 | Loss: 0.64061, Acc: 71.11% | Test Loss: 0.71310, Test Acc: 66.67%
Epoch: 110 | Loss: 0.61945, Acc: 73.33% | Test Loss: 0.69442, Test Acc: 73.33%
Epoch: 120 | Loss: 0.59996, Acc: 74.07% | Test Loss: 0.67712, T